In [3]:
pip install requests_cache

In [4]:
#librarys
import json
import requests
import numpy as np
import pandas as pd
import plotly.express as px
from requests_cache import CachedSession
from datetime import datetime

In [5]:
api_key = '4a8b15e8da98bcfed048039b4851248d'

In [6]:
def get_zip_weather(api_key,zipp, country = 'US'):
    """Get the weather for zip."""
    weather_url = f"https://api.openweathermap.org/data/2.5/weather?zip={zipp},{country}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

In [7]:
get_zip_weather(api_key,96729)

{'coord': {'lon': -157.0791, 'lat': 21.173},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 71.06,
  'feels_like': 70.68,
  'temp_min': 71.06,
  'temp_max': 72.34,
  'pressure': 1020,
  'humidity': 60},
 'visibility': 10000,
 'wind': {'speed': 16.11, 'deg': 40, 'gust': 26.46},
 'clouds': {'all': 40},
 'dt': 1702671958,
 'sys': {'type': 1,
  'id': 7870,
  'country': 'US',
  'sunrise': 1702659478,
  'sunset': 1702698546},
 'timezone': -36000,
 'id': 0,
 'name': 'Hoolehua',
 'cod': 200}

In [8]:
def State_info(state):
    url = f'https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-united-states-of-america-zc-point@public/records?limit=100&refine=stusps_code%3A%22{state}%22'
    session = CachedSession()
    response = session.get(url)
    return response.json()

In [9]:
State_info('DE')

{'total_count': 67,
 'results': [{'zip_code': '19944',
   'usps_city': 'Fenwick Island',
   'stusps_code': 'DE',
   'ste_name': 'Delaware',
   'zcta': 'TRUE',
   'parent_zcta': None,
   'population': 697.0,
   'density': 212.2,
   'primary_coty_code': '10005',
   'primary_coty_name': 'Sussex',
   'county_weights': '{"10005": "100"}',
   'coty_name': ['Sussex'],
   'cty_code': ['10005'],
   'imprecise': 'FALSE',
   'military': 'FALSE',
   'timezone': 'America/New_York',
   'geo_point_2d': {'lon': -75.05559, 'lat': 38.47578}},
  {'zip_code': '19953',
   'usps_city': 'Hartly',
   'stusps_code': 'DE',
   'ste_name': 'Delaware',
   'zcta': 'TRUE',
   'parent_zcta': None,
   'population': 4545.0,
   'density': 59.9,
   'primary_coty_code': '10001',
   'primary_coty_name': 'Kent',
   'county_weights': '{"10001": "100"}',
   'coty_name': ['Kent'],
   'cty_code': ['10001'],
   'imprecise': 'FALSE',
   'military': 'FALSE',
   'timezone': 'America/New_York',
   'geo_point_2d': {'lon': -75.69809, 

### Current Weather

In [10]:
def get_weather_map():
    api_key = '4a8b15e8da98bcfed048039b4851248d'
    try:
        state = str(input("Please enter the USPS state code (ie. California is CA): "))
        stateinfo = State_info(state)
        dfstateinfo = pd.DataFrame(stateinfo['results'])
    except ValueError:
        print("Invalid input. Please enter a valid state name. It is cap sensitive.")
    if stateinfo['total_count'] == 0:
        print("Invalid input. Please enter a valid state name. It is cap sensitive.")
    # county info
    print(set(dfstateinfo['primary_coty_name']))
    county = str(input(f'Please choose the county in {state} you would like to see from the list above:'))
    zipcodes = dfstateinfo['zip_code'][dfstateinfo['primary_coty_name'] == county]

    combined = []
    for zipcode in zipcodes:
        test_data = get_zip_weather(api_key,zipcode)
        #gather subseted data
        weather_info = test_data['weather'][0]
        main_info = test_data['main']
        wind_info = test_data['wind']
        loc_info = test_data['coord']
        city_info = test_data['name']
        # join all to a dictionary
        sub = {**{'name':city_info},
               **{'zip':zipcode},
               **weather_info,
               **main_info,
               **wind_info,
               **loc_info}
        combined.append(sub)
    df = pd.DataFrame(combined)
    df.head()


    fig = px.scatter_mapbox(df,
                        lon = df['lon'],
                        lat = df['lat'],
                        hover_name = df['name'],
                        hover_data = df[['feels_like','temp_min','temp_max','humidity']],
                        zoom = 5,
                        color = df['temp'],
                        width = 900,
                        height = 600,
                        size = df['speed'],
                        color_continuous_scale="temps",
                        title = f'Weather Map of {county} county')
    fig.update_layout(mapbox_style = 'carto-positron')
    fig.show()

In [11]:
# get_weather_map()

In [12]:
# state = str(input("Please enter the USPS state code (ie. California is CA): "))
# stateinfo = State_info(state)
# dfstateinfo = pd.DataFrame(stateinfo['results'])

In [13]:
# dfstateinfo

In [14]:
# zipcodes = dfstateinfo['zip_code'][dfstateinfo['primary_coty_name'] == 'Los Angeles']
zipcodes = [95616]
combined = []
for zipcode in zipcodes:
    test_data = get_zip_weather(api_key,zipcode)
    #gather subseted data
    weather_info = test_data['weather'][0]
    main_info = test_data['main']
    wind_info = test_data['wind']
    loc_info = test_data['coord']
    city_info = test_data['name']
    # join all to a dictionary
    sub = {**{'name':city_info},
            **{'zip':zipcode},
            **weather_info,
            **main_info,
            **wind_info,
            **loc_info}
    combined.append(sub)
df = pd.DataFrame(combined)
df.head()

,name,zip,id,main,description,icon,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level,speed,deg,gust,lon,lat
0,Davis,95616,804,Clouds,overcast clouds,04d,58.37,56.26,55.31,62.22,1022,50,1022,1020,7.67,357,8.7,-121.7418,38.5538


### Historical and Forecast

In [50]:
# https://history.openweathermap.org/data/2.5/history/city?q=London,UK&appid={API key}

#
def get_hist_weather(api_key, city, country = 'US'):
    """Get 24 hour historical weather for city name."""
    weather_url = f"https://history.openweathermap.org/data/2.5/history/city?q={city},{country}&cnt={24}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

def get_forecast_weather(api_key, city, country = 'US'):
    """Get 4 day hourly forecast for city name."""
    weather_url = f"https://pro.openweathermap.org/data/2.5/forecast/hourly?q={city},{country}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

In [51]:
hist_data = get_hist_weather(api_key, 'Davis')

forecast_data = get_forecast_weather(api_key, 'Davis')

In [52]:
hist_data

{'message': 'Count: 24',
 'cod': '200',
 'city_id': 5341704,
 'calctime': 0.018884874,
 'cnt': 24,
 'list': [{'dt': 1702580400,
   'main': {'temp': 51.31,
    'feels_like': 49.53,
    'pressure': 1025,
    'humidity': 72,
    'temp_min': 49.01,
    'temp_max': 54.99},
   'wind': {'speed': 0, 'deg': 0},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]},
  {'dt': 1702584000,
   'main': {'temp': 54.18,
    'feels_like': 52.39,
    'pressure': 1024,
    'humidity': 66,
    'temp_min': 51.33,
    'temp_max': 57.4},
   'wind': {'speed': 6.91, 'deg': 30},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]},
  {'dt': 1702587600,
   'main': {'temp': 56.52,
    'feels_like': 54.55,
    'pressure': 1024,
    'humidity': 57,
    'temp_min': 53.87,
    'temp_max': 59.22},
   'wind': {'speed': 0, 'deg': 0},
   'clouds': {'all': 0},
   'weather': 

In [53]:
# Build historical data frame
date_time = pd.DataFrame([pd.to_datetime(hist_data['list'][i]['dt'], unit='s') for i in range(len(hist_data['list']))])
main_data = pd.DataFrame([hist_data['list'][i]['main'] for i in range(len(hist_data['list']))])
wind_data = pd.DataFrame([hist_data['list'][i]['wind'] for i in range(len(hist_data['list']))])
cloud_data = pd.DataFrame([hist_data['list'][i]['clouds'] for i in range(len(hist_data['list']))])
weather_data = pd.DataFrame([hist_data['list'][i]['weather'][0] for i in range(len(hist_data['list']))])

hist_df = pd.concat([date_time, main_data, wind_data, cloud_data, weather_data], axis=1)
hist_df['data_type'] = "Historical"
hist_df

,0,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,gust,all,id,main,description,icon,data_type
0,2023-12-14 19:00:00,51.31,49.53,1025,72,49.01,54.99,0.00,0,NaN,0,800,Clear,clear sky,01d,Historical
1,2023-12-14 20:00:00,54.18,52.39,1024,66,51.33,57.40,6.91,30,NaN,0,800,Clear,clear sky,01d,Historical
2,2023-12-14 21:00:00,56.52,54.55,1024,57,53.87,59.22,0.00,0,NaN,0,800,Clear,clear sky,01d,Historical
3,2023-12-14 22:00:00,57.92,55.99,1023,55,54.88,61.21,10.36,10,NaN,0,800,Clear,clear sky,01d,Historical
4,2023-12-14 23:00:00,58.35,56.37,1023,53,55.87,61.20,8.05,20,NaN,0,800,Clear,clear sky,01d,Historical
5,2023-12-15 00:00:00,57.24,55.24,1023,55,55.87,58.57,8.05,30,NaN,0,800,Clear,clear sky,01d,Historical
6,2023-12-15 01:00:00,53.78,51.67,1023,60,51.15,55.98,0.00,0,NaN,0,800,Clear,clear sky,01n,Historical
7,2023-12-15 02:00:00,50.50,48.45,1023,68,47.53,52.99,0.00,0,NaN,0,800,Clear,clear sky,01n,Historical
8,2023-12-15 03:00:00,49.60,49.60,1023,69,46.40,52.00,0.00,0,NaN,0,800,Clear,clear sky,01n,Historical
9,2023-12-15 04:00:00,48.49,48.49,1023,71,45.36,51.67,0.00,0,NaN,0,800,Clear,clear sky,01n,Historical


In [54]:
forecast_data

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1702674000,
   'main': {'temp': 43.68,
    'feels_like': 43.68,
    'temp_min': 43.12,
    'temp_max': 43.68,
    'pressure': 1029,
    'sea_level': 1029,
    'grnd_level': 883,
    'humidity': 61,
    'temp_kf': 0.31},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 2.64, 'deg': 340, 'gust': 3.4},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2023-12-15 21:00:00'},
  {'dt': 1702677600,
   'main': {'temp': 43.61,
    'feels_like': 43.61,
    'temp_min': 43.3,
    'temp_max': 43.61,
    'pressure': 1029,
    'sea_level': 1029,
    'grnd_level': 882,
    'humidity': 59,
    'temp_kf': 0.17},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 5},
   'wind': {'speed': 1.74, 'deg': 217, 'gust': 1.79},
   'visibility': 10000,
   'po

In [55]:
# Build forecast data frame
date_time = pd.DataFrame([pd.to_datetime(forecast_data['list'][i]['dt'], unit='s') for i in range(len(forecast_data['list']))])
main_data = pd.DataFrame([forecast_data['list'][i]['main'] for i in range(len(forecast_data['list']))])
wind_data = pd.DataFrame([forecast_data['list'][i]['wind'] for i in range(len(forecast_data['list']))])
cloud_data = pd.DataFrame([forecast_data['list'][i]['clouds'] for i in range(len(forecast_data['list']))])
weather_data = pd.DataFrame([forecast_data['list'][i]['weather'][0] for i in range(len(forecast_data['list']))])


forecast_df = pd.concat([date_time, main_data, wind_data, cloud_data, weather_data], axis=1)
forecast_df['data_type'] = "Forecast"
forecast_df

,0,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,speed,deg,gust,all,id,main,description,icon,data_type
0,2023-12-15 21:00:00,43.68,43.68,43.12,43.68,1029,1029,883,61,0.31,2.64,340,3.40,0,800,Clear,clear sky,01d,Forecast
1,2023-12-15 22:00:00,43.61,43.61,43.30,43.61,1029,1029,882,59,0.17,1.74,217,1.79,5,800,Clear,clear sky,01d,Forecast
2,2023-12-15 23:00:00,42.96,39.72,41.86,42.96,1029,1029,882,61,0.61,5.32,304,5.14,9,800,Clear,clear sky,01d,Forecast
3,2023-12-16 00:00:00,40.93,38.62,39.09,40.93,1030,1030,882,64,1.02,3.76,338,4.05,13,801,Clouds,few clouds,02d,Forecast
4,2023-12-16 01:00:00,39.40,39.40,38.32,39.40,1031,1031,882,66,0.60,2.89,24,3.60,32,802,Clouds,scattered clouds,03n,Forecast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2023-12-19 16:00:00,44.31,44.31,44.31,44.31,1014,1014,869,59,0.00,1.90,115,2.24,100,804,Clouds,overcast clouds,04d,Forecast
92,2023-12-19 17:00:00,45.43,45.43,45.43,45.43,1014,1014,870,55,0.00,1.10,195,2.04,100,804,Clouds,overcast clouds,04d,Forecast
93,2023-12-19 18:00:00,45.64,45.64,45.64,45.64,1015,1015,870,53,0.00,2.80,237,3.15,100,804,Clouds,overcast clouds,04d,Forecast
94,2023-12-19 19:00:00,45.61,45.61,45.61,45.61,1014,1014,870,52,0.00,0.94,142,3.38,100,804,Clouds,overcast clouds,04d,Forecast


In [63]:
#merge data for plotting
plot_df = pd.concat([hist_df, forecast_df], ignore_index=True, sort=True, axis=0)
plot_df.rename(columns={plot_df.columns[0]: 'date_time'}, inplace=True)

In [64]:
plot_df

,date_time,all,data_type,deg,description,feels_like,grnd_level,gust,humidity,icon,id,main,pressure,sea_level,speed,temp,temp_kf,temp_max,temp_min
0,2023-12-14 19:00:00,0,Historical,0,clear sky,49.53,NaN,NaN,72,01d,800,Clear,1025,NaN,0.00,51.31,NaN,54.99,49.01
1,2023-12-14 20:00:00,0,Historical,30,clear sky,52.39,NaN,NaN,66,01d,800,Clear,1024,NaN,6.91,54.18,NaN,57.40,51.33
2,2023-12-14 21:00:00,0,Historical,0,clear sky,54.55,NaN,NaN,57,01d,800,Clear,1024,NaN,0.00,56.52,NaN,59.22,53.87
3,2023-12-14 22:00:00,0,Historical,10,clear sky,55.99,NaN,NaN,55,01d,800,Clear,1023,NaN,10.36,57.92,NaN,61.21,54.88
4,2023-12-14 23:00:00,0,Historical,20,clear sky,56.37,NaN,NaN,53,01d,800,Clear,1023,NaN,8.05,58.35,NaN,61.20,55.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023-12-19 16:00:00,100,Forecast,115,overcast clouds,44.31,869.0,2.24,59,04d,804,Clouds,1014,1014.0,1.90,44.31,0.0,44.31,44.31
116,2023-12-19 17:00:00,100,Forecast,195,overcast clouds,45.43,870.0,2.04,55,04d,804,Clouds,1014,1014.0,1.10,45.43,0.0,45.43,45.43
117,2023-12-19 18:00:00,100,Forecast,237,overcast clouds,45.64,870.0,3.15,53,04d,804,Clouds,1015,1015.0,2.80,45.64,0.0,45.64,45.64
118,2023-12-19 19:00:00,100,Forecast,142,overcast clouds,45.61,870.0,3.38,52,04d,804,Clouds,1014,1014.0,0.94,45.61,0.0,45.61,45.61


In [67]:
#TODO: Plot historical and forecast data on same plot. Include lines for temp, max temp, min temp. Include hover option for other measures like wind speed, weather description, feels like.


# Create the plot
fig = px.line(plot_df, x='date_time', y=['temp_min', 'temp_max', 'temp'], custom_data=['speed', 'description'],
              labels={'date_time': 'Date','value': 'Temperature (°F)', 'variable': 'Temperature Type'},
              title='Historical and Forecast Temperature Data')

# Add hover data
fig.update_traces(mode='lines+markers')
for trace in fig.data:
    trace.name = trace.name.replace('_',' ').title()
fig.update_layout(hovermode='x unified')
# fig.add_trace(
#     px.scatter(plot_df, x='date_time', y='temp', custom_data=['speed', 'description']).data[0]
# )
fig.update_traces(hovertemplate="<br>".join([
    "%{y}°F",
    "Wind Speed: %{customdata[0]} mph",
    "Weather: %{customdata[1]}",
]))

# Show the plot
fig.show()